<a href="https://colab.research.google.com/github/ngoan22mse23088/python4engineers-assignments/blob/master/Assignments2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python
!apt-get install mysql-server > /dev/null
!service mysql start

!mysql -u root -e "SELECT 'Connected';"
!mysql -u root -e "CREATE DATABASE mydatabase;"
!mysql -u root -e "CREATE USER 'root'@'localhost' IDENTIFIED BY 'root';"
!mysql -u root -e "GRANT ALL PRIVILEGES ON mydatabase.* TO 'root'@'localhost';"
!mysql -u root -e "SELECT User FROM mysql.user;"
!mysql -u root -e "SHOW DATABASES;"
!mysql -u root -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'root';"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
W: Operation was interrupted before it could finish
^C
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.
Enter password: 

In [4]:
import mysql.connector

# Kết nối đến database vừa tạo
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="root",
  database="mydatabase"
)

# Tạo database
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE posts (id int NOT NULL AUTO_INCREMENT,created TIMESTAMP DEFAULT CURRENT_TIMESTAMP NOT NULL,title varchar(255),content varchar(255),PRIMARY KEY (id));")
mycursor.execute("SELECT * FROM posts")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)
print("completed")

DatabaseError: ignored

In [2]:
!pip install flask
!pip install colabcode
!pip install flask_mysqldb
!pip install mysql-connector-python
!pip install flask-ngrok
!pip install pyngrok
!ngrok http 5000

Traceback (most recent call last):
  File "/usr/lib/python3.9/subprocess.py", line 1189, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.9/subprocess.py", line 1917, in _wait
    (pid, sts) = self._try_wait(0)
  File "/usr/lib/python3.9/subprocess.py", line 1875, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/ngrok.py", line 517, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/ngrok.py", line 505, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.9/dist-packages/pyngrok/process.py", line 347, in run_process
    subprocess.call(start)
  File "/usr/lib/python3.9/subprocess.py", line 351, in call
    return p.wait(timeout=t

In [ ]:
!ngrok config add-authtoken 2NwsOLXovIUtXLuIqCCHXtJedAo_4cA58oGig9qeTAeqWiguh

In [ ]:
from flask import Flask
from pyngrok import ngrok

app = Flask(__name__)

public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")

@app.route("/")
def home():    
    return "Hello, World!"

if __name__ == "__main__":
    app.run()

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")

In [6]:
from flask import Flask, jsonify, request
from flask_ngrok import run_with_ngrok
import mysql.connector

app = Flask(__name__)

run_with_ngrok(app) 

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="root",
    database="mydatabase"
)

@app.route('/posts', methods=['GET'])
def get_posts():
    cursor = db.cursor()
    cursor.execute("SELECT * FROM posts")
    result = cursor.fetchall()
    posts = []
    for post in result:
        posts.append({
            'id': post[0],
            'created': post[1],
            'title': post[2],
            'content': post[3]
        })
    return jsonify(posts)

@app.route('/posts', methods=['POST'])
def create_post():
    cursor = db.cursor()
    title = request.json['title']
    content = request.json['content']
    cursor.execute("INSERT INTO posts (title, content) VALUES (%s, %s)", (title, content))
    db.commit()
    return jsonify({'message': 'Post created successfully'})

@app.route('/posts/<int:id>', methods=['GET'])
def get_post(id):
    cursor = db.cursor()
    cursor.execute("SELECT * FROM posts WHERE id = %s", (id,))
    result = cursor.fetchone()
    if result is None:
        return jsonify({'message': 'Post not found'}), 404
    post = {
        'id': result[0],
        'created': result[1],
        'title': result[2],
        'content': result[3]
    }
    return jsonify(post)

@app.route('/posts/<int:id>', methods=['PUT'])
def update_post(id):
    cursor = db.cursor()
    title = request.json['title']
    content = request.json['content']
    cursor.execute("UPDATE posts SET title = %s, content = %s WHERE id = %s", (title, content, id))
    db.commit()
    return jsonify({'message': 'Post updated successfully'})

@app.route('/posts/<int:id>', methods=['DELETE'])
def delete_post(id):
    cursor = db.cursor()
    cursor.execute("DELETE FROM posts WHERE id = %s", (id,))
    db.commit()
    return jsonify({'message': 'Post deleted successfully'})

if __name__ == '__main__':
    # app.run()
    app.run(debug=True, port=8181)


DatabaseError: ignored